In [1]:
import datajoint as dj
from ibl_pipeline import behavior, ephys
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from ibl_pipeline.plotting import plotting_utils_ephys as putils
import scipy.signal as signal

Connecting shan@datajoint.internationalbrainlab.org:3306


In [2]:
def compute_psth(trials, trial_type, align_event, nbins, window_size, x_lim=[-1, 1]):
    
    if trial_type == 'left':
        color = 'green'
    elif trial_type == 'right':
        color = 'blue'
    elif trial_type == 'all':
        color = 'black'
    elif trial_type == 'incorrect':
        color = 'red'
    else:
        raise NameError('Invalid type name')
        
    spk_times = (trials & 'event="{}"'.format(align_event)).fetch('trial_spike_times')
    mean_counts = np.divide(
        np.histogram(np.hstack(spk_times),
                     range=x_lim,
                     bins=nbins)[0],
        len(spk_times))
    time_bins=np.linspace(x_lim[0], x_lim[1], num=nbins)
    
    # convolve with a box-car filter
    dt = np.mean(np.diff(time_bins))
    psth = np.divide(signal.convolve(mean_counts, signal.boxcar(window_size), mode='same'),
                     window_size*dt)
    
    data = go.Scatter(
        x=list(time_bins),
        y=list(psth),
        mode='lines',
        marker=dict(
            size=6,
            color=color),
        name='{} trials'.format(trial_type)
    )
    
    return data

In [ ]:
trials

In [8]:
cluster = ephys.Cluster & 'cluster_id=1'
key = (ephys.Event & 'event = "stim on"').fetch('KEY')
trials_all = (behavior.TrialSet.Trial * ephys.TrialSpikes & cluster).proj(
    'trial_start_time', 'trial_stim_on_time', 'trial_response_time', 'trial_feedback_time',
    'trial_response_choice', 'trial_spike_times',
    trial_duration='trial_end_time-trial_start_time',
    trial_signed_contrast='trial_stim_contrast_right - trial_stim_contrast_left'
) & 'trial_duration < 5' & 'trial_response_choice!="No Go"' & key

trials_left = trials_all & 'trial_response_choice="CW"' & 'trial_signed_contrast < 0'
trials_right = trials_all & 'trial_response_choice="CCW"' & 'trial_signed_contrast > 0'
trials_incorrect = trials_all - trials_right.proj() - trials_left.proj()

align_event = (ephys.Event & key).fetch('event')
x_lim = [-1, 1]
data = []
if len(trials_left):
    data.append(
        putils.compute_psth(trials_left, 'left', align_event, 1000, 10, x_lim)
    )
if len(trials_right):
    data.append(
        putils.compute_psth(trials_right, 'right', align_event, 1000, 10, x_lim)
    )
if len(trials_incorrect):
    data.append(
        putils.compute_psth(trials_incorrect, 'incorrect', align_event, 1000, 10, x_lim)
    )

data.append(
    putils.compute_psth(trials_all, 'all', align_event, 1000, 10, x_lim)
)

layout = go.Layout(
    width=580,
    height=370,
    margin=go.layout.Margin(
        l=50,
        r=30,
        b=40,
        t=80,
        pad=0
    ),
    title=dict(
        text='PSTH, aligned to {} time'.format(align_event),
        x=0.2,
        y=0.87
    ),
    xaxis=dict(
        title='Time (sec)',
        range=x_lim,
        showgrid=False
    ), 
    yaxis=dict(
        title='Firing rate (spks/sec)',
        showgrid=False
    ),
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:169: UserWarning:

Mysql server has gone away. Reconnecting to the server.



ValueError: need at least one array to concatenate

In [10]:
align_event

array(['stim on'], dtype=object)

In [9]:
trials_left

subject_uuid,session_start_time start time,trial_id trial identification number,probe_idx probe insertion number,cluster_revision,cluster_id,event,trial_start_time beginning of quiescent period time (seconds),trial_response_time,trial_response_choice which choice was made in choiceworld,trial_stim_on_time Time of stimulus in choiceworld (seconds),trial_feedback_time Time of feedback delivery (reward or not) in choiceworld,"trial_spike_times spike time for each trial, aligned to go cue time",trial_duration calculated attribute,trial_signed_contrast calculated attribute
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,6,0,0,1,stim on,136.8624,0.381533,CW,136.9231,140.56083333333333,=BLOB=,4.698433333333327,-1.0
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,7,0,0,1,stim on,141.95523333333333,0.455433,CW,142.02266666666668,143.55326666666667,=BLOB=,2.5980333333333476,-0.125
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,12,0,0,1,stim on,164.0096,nan,CW,164.07146666666668,164.9186,=BLOB=,1.9089999999999918,-0.25
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,22,0,0,1,stim on,199.31366666666668,0.320667,CW,199.3694,201.3052,=BLOB=,2.9915333333333365,-0.25
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,26,0,0,1,stim on,211.83356666666666,0.713133,CW,211.90186666666668,213.88206666666667,=BLOB=,3.0485000000000184,-0.125
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,32,0,0,1,stim on,233.00783333333334,0.281933,CW,233.06733333333332,234.21383333333333,=BLOB=,2.205999999999989,-0.0625
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,35,0,0,1,stim on,249.20513333333332,0.336067,CW,249.26636666666667,250.01516666666666,=BLOB=,1.8100333333333367,-0.0625
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,37,0,0,1,stim on,254.3306,0.458033,CW,254.3992,256.1442,=BLOB=,2.813600000000008,-0.0625
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,45,0,0,1,stim on,299.1107333333333,0.4668,CW,299.17993333333334,301.07463333333334,=BLOB=,2.9639000000000237,-0.125
efa5e878-6d7a-47ef-8ec8-ac7d6272cf22,2019-05-07 17:22:20,47,0,0,1,stim on,305.1831666666667,0.3,CW,305.2463666666667,307.2262,=BLOB=,3.0430333333333124,-0.125


In [5]:
import json
f = open("psth_combined.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()